In [4]:
import sys
sys.path.append('/Users/tgn/ConfigArgParse-0.10.0/build/lib')
sys.path.append('/Users/tgn/neon/build/lib')

In [14]:
from neon.backends import gen_backend
be = gen_backend(backend='cpu', batch_size=100)

In [17]:
import numpy as np
from neon import logger as neon_logger
from neon.data import ArrayIterator, load_mnist
from neon.initializers import Uniform
from neon.layers import Conv, Pooling, GeneralizedCost, Deconv, Affine
from neon.models import Model
from neon.optimizers import GradientDescentMomentum
from neon.transforms import Rectlin, SumSquared
from neon.callbacks.callbacks import Callbacks
from neon.util.argparser import NeonArgparser

# parse the command line arguments


# Load dataset
(X_train, y_train), (X_test, y_test), nclass = load_mnist()

X_train=X_train[0:100]

# Set input and target to X_train
train = ArrayIterator(X_train)

# Initialize the weights and the learning rule
init_uni = Uniform(low=-0.1, high=0.1)
opt_gdm = GradientDescentMomentum(learning_rate=0.001, momentum_coef=0.9)



In [26]:
# Strided conv autoencoder
bn = False
layers = [Affine(nout=28*14, init=init_uni, activation=Rectlin(), batch_norm=bn),
          Affine(nout=28*7, init=init_uni, activation=Rectlin(), batch_norm=bn),
          Affine(nout=28*14, init=init_uni, activation=Rectlin(), batch_norm=bn),
         Affine(nout=28*28, init=init_uni, batch_norm=bn)]

# Define the cost
cost = GeneralizedCost(costfunc=SumSquared())

model = Model(layers=layers)

# configure callbacks
callbacks = Callbacks(model, train)

# Fit the model
model.fit(train, optimizer=opt_gdm, num_epochs=200,
          cost=cost, callbacks=callbacks)


Epoch 0   [Train |████████████████████|    1/1    batches, 68.35 cost, 0.04s]
Epoch 1   [Train |████████████████████|    1/1    batches, 63.38 cost, 0.03s]
Epoch 2   [Train |████████████████████|    1/1    batches, 58.77 cost, 0.04s]
Epoch 3   [Train |████████████████████|    1/1    batches, 55.39 cost, 0.04s]
Epoch 4   [Train |████████████████████|    1/1    batches, 52.98 cost, 0.02s]
Epoch 5   [Train |████████████████████|    1/1    batches, 51.21 cost, 0.03s]
Epoch 6   [Train |████████████████████|    1/1    batches, 49.85 cost, 0.04s]
Epoch 7   [Train |████████████████████|    1/1    batches, 48.77 cost, 0.05s]
Epoch 8   [Train |████████████████████|    1/1    batches, 47.87 cost, 0.02s]
Epoch 9   [Train |████████████████████|    1/1    batches, 47.10 cost, 0.06s]
Epoch 10  [Train |████████████████████|    1/1    batches, 44.23 cost, 0.02s]
Epoch 11  [Train |████████████████████|    1/1    batches, 42.31 cost, 0.05s]
Epoch 12  [Train |████████████████████|    1/1    batches, 41.21

In [27]:
# Plot the reconstructed digits
try:
    from matplotlib import pyplot, cm
    fi = 0
    nrows = 10
    ncols = 10
    test = np.zeros((28 * nrows, 28 * ncols))
    idxs = [(row, col) for row in range(nrows) for col in range(ncols)]
    for row, col in idxs:
        im = model.layers.layers[-1].outputs.get()[:, fi].reshape((28, 28))
        test[28 * row:28 * (row + 1):, 28 * col:28 * (col + 1)] = im
        fi = fi + 1
    pyplot.matshow(test, cmap=cm.gray)
    pyplot.savefig('Reconstructed.png')
except ImportError:
    neon_logger.display(
        'matplotlib needs to be manually installed to generate plots')

In [72]:
im = model.layers.layers[-2].outputs.get()[:, :].T
print im.shape

(100, 392)


In [73]:
from time import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix,accuracy_score, f1_score, precision_score, recall_score

In [74]:
labels = y_train[0:100]
sample_size = 100

In [101]:
def bench_k_means(estimator, name, data):
    t0 = time()
    estimator.fit(data)
    print('name   time   estimator   homogeneity   completeness    v_measure   adjusted_rand   adjusted_mutual_info   silhouette\n% 9s   %.2fs    %i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
          % (name, (time() - t0), estimator.inertia_,
             metrics.homogeneity_score(labels, estimator.labels_),
             metrics.completeness_score(labels, estimator.labels_),
             metrics.v_measure_score(labels, estimator.labels_),
             metrics.adjusted_rand_score(labels, estimator.labels_),
             metrics.adjusted_mutual_info_score(labels,  estimator.labels_),
             metrics.silhouette_score(data, estimator.labels_,
                                      metric='euclidean',
                                      sample_size=sample_size)))

In [102]:
bench_k_means(KMeans(init='k-means++', n_clusters=10, n_init=10),
              name="k-means++", data=im)
bench_k_means(KMeans(init='k-means++', n_clusters=10, n_init=10),
              name="k-means++", data=X_train)

name   time   estimator   homogeneity   completeness    v_measure   adjusted_rand   adjusted_mutual_info   silhouette
k-means++   0.06s    178   0.633   0.643   0.638   0.408   0.536    0.221
name   time   estimator   homogeneity   completeness    v_measure   adjusted_rand   adjusted_mutual_info   silhouette
k-means++   0.05s    3004   0.638   0.650   0.644   0.394   0.542    0.105


In [103]:
kmeans = KMeans(init='k-means++', n_clusters=10, n_init=10)
kmeans.fit(im)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [104]:
Z = kmeans.predict(im)
print Z
print labels

[6 7 0 2 3 3 8 5 8 3 5 8 5 1 8 3 0 6 0 3 0 7 3 2 8 5 8 5 9 2 0 6 0 3 4 8 1
 4 0 1 8 6 8 3 0 3 6 6 0 5 0 7 9 2 0 6 7 3 9 2 0 2 1 7 0 2 1 2 0 4 8 3 8 1
 5 7 0 2 2 3 0 4 4 1 0 6 0 3 7 3 1 3 0 1 6 4 3 6 5 2]
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9 4 0 9 1 1 2 4 3 2 7 3 8 6 9 0 5 6
 0 7 6 1 8 7 9 3 9 8 5 9 3 3 0 7 4 9 8 0 9 4 1 4 4 6 0 4 5 6 1 0 0 1 7 1 6
 3 0 2 1 1 7 9 0 2 6 7 8 3 9 0 4 6 7 4 6 8 0 7 8 3 1]


In [105]:
def plot_confusion_matrix(cm, classlist1, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classlist1))
    plt.xticks(tick_marks, classlist1, rotation=45)
    plt.yticks(tick_marks, classlist1)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [80]:
cm = confusion_matrix(labels, Z)

In [106]:
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
classlist1=[0,1,2,3,4,5,6,7,8,9]
plt.figure()
plot_confusion_matrix(cm_normalized,classlist1, title='Normalized confusion matrix')
plt.savefig('cm_mnist.png', format='png', dpi=600)
plt.show()

In [82]:
# Step size of the mesh. Decrease to increase the quality of the VQ.
h = .02     # point in the mesh [x_min, m_max]x[y_min, y_max].

# Plot the decision boundary. For that, we will assign a color to each
x_min, x_max = reduced_data[:, 0].min() - 1, reduced_data[:, 0].max() + 1
y_min, y_max = reduced_data[:, 1].min() - 1, reduced_data[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Obtain labels for each point in mesh. Use last trained model.
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1)
plt.clf()
plt.imshow(Z, interpolation='nearest',
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap=plt.cm.Paired,
           aspect='auto', origin='lower')

plt.plot(reduced_data[:, 0], reduced_data[:, 1], 'k.', markersize=2)
# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(centroids[:, 0], centroids[:, 1],
            marker='x', s=169, linewidths=3,
            color='w', zorder=10)
plt.title('K-means clustering on the digits dataset (PCA-reduced data)\n'
          'Centroids are marked with white cross')
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()


ValueError: Incorrect number of features. Got 2 features, expected 392